# Imports

In [1]:
## Logging and Shell
import logging
logging.basicConfig(
    level=logging.ERROR,
    force=True
)

## Numeric imports
import pandas as pd

## File I/O
from pathlib import Path
import json

# Cheminformatics
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

DIM    = 300
ASPECT = 3/2
IPythonConsole.molSize = (int(ASPECT*DIM), DIM)   # Change image size

# Custom imports
from polymerist.monomers import specification
from polymerist.rdutils.reactions import reactions

# Static Paths
RAW_DATA_DIR  = Path('monomer_data_raw')
FMT_DATA_DIR  = Path('monomer_data_formatted')
PROC_DATA_DIR = Path('monomer_data_processed')
RXN_FILES_DIR = Path('poly_rxns')

PDB_OUT_DIR   = Path('pdb_files')
PDB_OUT_DIR.mkdir(exist_ok=True)

MONO_INFO_DIR = Path('monomer_fragments')
MONO_INFO_DIR.mkdir(exist_ok=True)

# Load monomer and rxn data 

In [4]:
input_data_path = PROC_DATA_DIR / '20231114_polyid_data_density_DP2-6 - 1,2 monomers_FILTERED.csv'
# input_data_path = PROC_DATA_DIR / 'nipu_urethanes_FILTERED.csv'
df = pd.read_csv(input_data_path, index_col=[0,1])
df

smiles_monomer_0  \
mechanism              polymer_name                                                                                            
polycarbonate_phosgene poly(4-[1,1,1,3,3,3-hexafluoro-2-(4-hydroxyphen...  [H]-[O]-[C]1=[C](-[H])-[C](-[H])=[C](-[C](-[C]...   
                       poly(4-[2-(4-hydroxy-3,5-dimethylphenyl)propan-...  [H]-[O]-[C]1=[C](-[C](-[H])(-[H])-[H])-[C](-[H...   
polyimide              poly(4-[9-(4-aminophenyl)fluoren-9-yl]aniline-c...  [H]-[C]1=[C](-[H])-[C](-[H])=[C]2-[C](=[C]-1-[...   
                       poly(benzene-1,3-diamine-co-5-[3-tert-butyl-5-(...  [H]-[C]1=[C](-[H])-[C](-[N](-[H])-[H])=[C](-[H...   
                       poly(4-[(4-aminophenyl)methyl]aniline-co-5-[2-(...  [H]-[C]1=[C](-[N](-[H])-[H])-[C](-[H])=[C](-[H...   
...                                                                                                                      ...   
                       poly(4-[4-[2-[4-(4-aminophenoxy)phenyl]propan-2...  [H]-[C]1=[C](-[O]-[C]2=[C](-[H])-[C](-[H])=[C]...   
                       poly(4-[2-(4-amino-3-methylphenyl)-1,1,1,3,3,3-...  [H]-[C]1=[C](-[N](-[H])-[H])-[C](-[C](-[H])(-[...   
                       poly(2,4,6-trimethylbenzene-1,3-diamine-co-5-(1...  [H]-[C]1=[C](-[C](-[H])(-[H])-[H])-[C](-[N](-[...   
polycarbonate_phosgene poly(4-[1-(4-hydroxyphenyl)cycloheptyl]phenol-c...  [H]-[O]-[C]1=[C](-[H])-[C](-[H])=[C](-[C]2(-[C...   
polyester              poly(butane-1,4-diol-co-hexanedioic acid)           [H]-[O]-[C](-[H])(-[H])-[C](-[H])(-[H])-[C](-[...   

                                                                          fn_group_name_monomer_0  \
mechanism              polymer_name                                                                 
polycarbonate_phosgene poly(4-[1,1,1,3,3,3-hexafluoro-2-(4-hydroxyphen...                hydroxyl   
                       poly(4-[2-(4-hydroxy-3,5-dimethylphenyl)propan-...                hydroxyl   
polyimide              poly(4-[9-(4-aminophenyl)fluoren-9-yl]aniline-c...                   amine   
                       poly(benzene-1,3-diamine-co-5-[3-tert-butyl-5-(...                   amine   
                       poly(4-[(4-aminophenyl)methyl]aniline-co-5-[2-(...                   amine   
...                                                                                           ...   
                       poly(4-[4-[2-[4-(4-aminophenoxy)phenyl]propan-2...                   amine   
                       poly(4-[2-(4-amino-3-methylphenyl)-1,1,1,3,3,3-...                   amine   
                       poly(2,4,6-trimethylbenzene-1,3-diamine-co-5-(1...                   amine   
polycarbonate_phosgene poly(4-[1-(4-hydroxyphenyl)cycloheptyl]phenol-c...                hydroxyl   
polyester              poly(butane-1,4-diol-co-hexanedioic acid)                         hydroxyl   

                                                                                                        IUPAC_name_monomer_0  \
mechanism              polymer_name                                                                                            
polycarbonate_phosgene poly(4-[1,1,1,3,3,3-hexafluoro-2-(4-hydroxyphen...  4-[1,1,1,3,3,3-hexafluoro-2-(4-hydroxyphenyl)p...   
                       poly(4-[2-(4-hydroxy-3,5-dimethylphenyl)propan-...  4-[2-(4-hydroxy-3,5-dimethylphenyl)propan-2-yl...   
polyimide              poly(4-[9-(4-aminophenyl)fluoren-9-yl]aniline-c...           4-[9-(4-aminophenyl)fluoren-9-yl]aniline   
                       poly(benzene-1,3-diamine-co-5-[3-tert-butyl-5-(...                                benzene-1,3-diamine   
                       poly(4-[(4-aminophenyl)methyl]aniline-co-5-[2-(...                   4-[(4-aminophenyl)methyl]aniline   
...                                                                                                                      ...   
                       poly(4-[4-[2-[4-(4-aminophenoxy)phenyl]propan-2...  4-[4-[2-[4-(4-aminophenoxy)phe

In [3]:
blacklisted_rxns = ['imide']#, 'vinyl']
df = df[df.mechanism.map(lambda s : s not in blacklisted_rxns)]

AttributeError: 'DataFrame' object has no attribute 'mechanism'

In [ ]:
keys = ['rxn_name']

df_grouper = df.groupby(keys)
frames = {
    mech : df_grouper.get_group(mech)
        for mech in df_grouper.groups
}

## Load pre-defined reactions with functional group and name backmap

In [ ]:
with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

rxns = {
    rxnname : reactions.AnnotatedReaction.from_rxnfile(RXN_FILES_DIR / f'{rxnname}.rxn')
        for rxnname in rxn_groups.keys()
}

# Polymerizing fragments

## Example for polymerization of individual molecule

In [ ]:
mechname = 'polyurethane_isocyanate'
mechname = 'polyamide'
frame = frames[mechname]
row = frame.loc[frame.index[0]]
rxn_pathway = rxns[mechname]

named_reactants = {}
for i in range(2):
    reactant = Chem.MolFromSmiles(row[f'smiles_monomer_{i}'], sanitize=False)
    Chem.SanitizeMol(reactant, sanitizeOps=specification.SANITIZE_AS_KEKULE)
    display(reactant)
    named_reactants[ row[f'IUPAC_name_monomer_{i}'] ] = reactant

reactants = [i for i in named_reactants.values()]

In [ ]:
from polymerist.rdutils.bonding import portlib
from polymerist.rdutils.reactions import reactors

display(rxn_pathway)
reactor = reactors.PolymerizationReactor(rxn_pathway)
for (dimer, pair) in reactor.propagate(reactants):
    display(dimer)
    for frag in pair:
        display(frag)

    print('='*50)